In [ ]:
!pip install altair

In [ ]:
###############################################
# 1. Imports & Basic Setup
###############################################
%%capture
import numpy as np
import pandas as pd
import requests
from io import StringIO
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score
from sklearn.feature_selection import mutual_info_classif
import warnings
warnings.filterwarnings('ignore')
from ipywidgets import interact, IntSlider, FloatSlider, Button, VBox
from tqdm import tqdm
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
import altair as alt

In [ ]:
###############################################
# 2. Load & Preprocess the Adult Dataset
###############################################

def load_adult_data(url='https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data'):
    """
    Load and preprocess the Adult dataset.
    """
    columns = ["age", "workclass", "fnlwgt", "education", "education_num",
               "marital_status", "occupation", "relationship", "race", "sex",
               "capital_gain", "capital_loss", "hours_per_week", "native_country",
               "income"]
    response = requests.get(url)
    data_str = response.text
    df = pd.read_csv(StringIO(data_str), header=None, names=columns,
                     na_values=' ?', skipinitialspace=True)
    df.dropna(inplace=True)
    for c in df.select_dtypes(include=['object']):
        df[c] = df[c].str.strip()
    df['income'] = df['income'].apply(lambda x: 1 if x == '>50K' else 0)
    return df

df_original = load_adult_data()

# Use only the specified QI columns.
# QI_cols = ['age', 'workclass', 'education_num',
#        'marital_status', 'race', 'sex',
#         'hours_per_week', 'native_country']
# QI_cols = ['age','sex','race','workclass']
QI_cols = ['age', 'workclass', 'education_num',
        'race', 'sex','hours_per_week']
sensitive_attribute = 'income'

# Compute non-QI columns (all columns except QI and the sensitive attribute).
non_QI_cols = [col for col in df_original.columns if col not in (QI_cols + [sensitive_attribute])]

In [ ]:
df_original.columns

Index(['age', 'workclass', 'fnlwgt', 'education', 'education_num',
       'marital_status', 'occupation', 'relationship', 'race', 'sex',
       'capital_gain', 'capital_loss', 'hours_per_week', 'native_country',
       'income'],
      dtype='object')

In [ ]:
df_original.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32561 entries, 0 to 32560
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             32561 non-null  int64 
 1   workclass       32561 non-null  object
 2   fnlwgt          32561 non-null  int64 
 3   education       32561 non-null  object
 4   education_num   32561 non-null  int64 
 5   marital_status  32561 non-null  object
 6   occupation      32561 non-null  object
 7   relationship    32561 non-null  object
 8   race            32561 non-null  object
 9   sex             32561 non-null  object
 10  capital_gain    32561 non-null  int64 
 11  capital_loss    32561 non-null  int64 
 12  hours_per_week  32561 non-null  int64 
 13  native_country  32561 non-null  object
 14  income          32561 non-null  int64 
dtypes: int64(7), object(8)
memory usage: 3.7+ MB


In [ ]:
df_original.income.mean()

0.2408095574460244

In [ ]:
###############################################
# 3. Helper Functions for Equivalence Classes
###############################################

def create_equivalence_classes(df, QI_cols):
    """
    Create an equivalence class identifier for each row using only the QI_cols.
    """
    eq_class_id = df[QI_cols].apply(lambda row: tuple(row), axis=1)
    return eq_class_id

def get_eq_class_sizes(eq_class_id):
    """
    Return a dictionary mapping each equivalence class to its size.
    """
    sizes = eq_class_id.value_counts().to_dict()
    return sizes

In [ ]:
###############################################
# 4. Generalization of QI Columns (Only for the given QIs)
###############################################

def generalize_QIs(df, QI_cols = QI_cols):
    """
    Generalize quasi-identifier columns in the dataset based on the given QI columns.
    """
    df = df.copy()

    if 'age' in QI_cols:
        # Generalize 'age'
        df['age'] = pd.cut(
            df['age'],
            bins=[0, 25, 45, 65, 100],
            labels=['Young', 'Adult', 'Senior', 'Elderly'],
            right=False
        )

    if 'sex' in QI_cols:
        # Generalize 'sex'
        df['sex'] = df['sex'].astype('category')

    if 'race' in QI_cols:
        # Generalize 'race'
        def generalize_race(race):
            if race in ['White', 'Black']:
                return race
            return 'Other'
        df['race'] = df['race'].apply(generalize_race)

    if 'education_num' in QI_cols:
        # Generalize 'education_num'
        df['education_num'] = pd.cut(
            df['education_num'],
            bins=[0, 5, 10, 14, 16],
            labels=['Low', 'Mid', 'High', 'Top'],
            include_lowest=True,
            right=True
        )

    if 'native_country' in QI_cols:
        # Generalize 'native_country'
        def generalize_native_country(country):
            continent_mapping = {
                'North America': ['United-States', 'Canada', 'Mexico'],
                'South America': ['Columbia', 'Peru', 'Ecuador'],
                'Europe': ['England', 'Germany', 'Italy', 'Poland', 'Portugal',
                           'France', 'Greece', 'Ireland', 'Scotland'],
                'Asia': ['China', 'Japan', 'India', 'Iran', 'Philippines',
                         'Vietnam', 'Cambodia', 'Laos', 'Thailand', 'Taiwan'],
                'Other': ['Outlying-US(Guam-USVI-etc)', 'Puerto-Rico', 'Honduras',
                          'Jamaica', 'Cuba', 'Dominican-Republic', 'El-Salvador',
                          'Guatemala', 'Haiti', 'Nicaragua', 'Trinadad&Tobago',
                          'South', 'Hong', 'Yugoslavia', 'Hungary',
                          'Holand-Netherlands']
            }
            for continent, countries in continent_mapping.items():
                if country in countries:
                    return continent
            return 'Other'
        df['native_country'] = df['native_country'].apply(generalize_native_country)

    if 'hours_per_week' in QI_cols:
        # Generalize 'hours_per_week'
        def generalize_hours_per_week(hours):
            if hours <= 20:
                return '0-20'
            elif hours <= 40:
                return '21-40'
            elif hours <= 60:
                return '41-60'
            return '61+'
        df['hours_per_week'] = df['hours_per_week'].apply(generalize_hours_per_week)

    if 'workclass' in QI_cols:
        # Generalize 'workclass'
        def generalize_workclass(workclass):
            if workclass in ['Private', 'Self-emp-not-inc', 'Self-emp-inc']:
                return 'Private/Self-Employed'
            elif workclass in ['Federal-gov', 'Local-gov', 'State-gov']:
                return 'Government'
            return 'Other/Unknown'
        df['workclass'] = df['workclass'].apply(generalize_workclass)

    if 'marital_status' in QI_cols:
        # Generalize 'marital_status'
        def generalize_marital_status(status):
            if status in ['Never-married', 'Married-civ-spouse', 'Married-AF-spouse']:
                return 'Not Divorced'
            return 'Divorced/Separated'
        df['marital_status'] = df['marital_status'].apply(generalize_marital_status)

    return df


In [ ]:
###############################################
# 5. Naïve Anonymization Approaches (Fixed Generalization & Suppression)
###############################################

def suppress_values(df, mask, QI_cols):
    """
    For each column in QI_cols, if the column is categorical, add "Suppressed"
    to its categories (if not already present) before setting the suppressed values.
    Then, set the QI values for rows where mask is True to "Suppressed".
    """
    for col in QI_cols:
        if pd.api.types.is_categorical_dtype(df[col]):
            if "Suppressed" not in df[col].cat.categories:
                df[col] = df[col].cat.add_categories(["Suppressed"])
        df.loc[mask, col] = "Suppressed"
    return df

def k_anonymity(df, QI_cols, k):
    """
    Enforce k-anonymity on the given QI columns.
    Generalize using fixed rules and, if an equivalence class has fewer than k records,
    suppress its QI values.
    """
    df_temp = generalize_QIs(df)
    eq_ids = create_equivalence_classes(df_temp, QI_cols)
    sizes = get_eq_class_sizes(eq_ids)
    if min(sizes.values()) >= k:
        return df_temp
    else:
        to_suppress = [eq for eq, size in sizes.items() if size < k]
        mask = create_equivalence_classes(df_temp, QI_cols).apply(lambda x: x in to_suppress)
        df_temp = suppress_values(df_temp, mask, QI_cols)
        return df_temp

def l_diversity(df, QI_cols, sensitive_col, l):
    """
    Enforce l-diversity on the given QI columns using fixed generalization.
    If an equivalence class does not have at least l distinct sensitive values,
    suppress its QI values.
    """
    df_temp = generalize_QIs(df)
    eq_ids = create_equivalence_classes(df_temp, QI_cols)
    diversity = df_temp.groupby(eq_ids)[sensitive_col].nunique()
    if all(diversity >= l):
        return df_temp
    else:
        to_suppress = [eq for eq, d in diversity.items() if d < l]
        mask = create_equivalence_classes(df_temp, QI_cols).apply(lambda x: x in to_suppress)
        df_temp = suppress_values(df_temp, mask, QI_cols)
        return df_temp

def t_closeness(df, QI_cols, sensitive_col, t):
    """
    Enforce t-closeness on the given QI columns using fixed generalization.
    For each equivalence class, if the L1 distance between its sensitive distribution
    and the global sensitive distribution exceeds t, suppress its QI values.
    """
    df_temp = generalize_QIs(df)
    global_dist = df[sensitive_col].value_counts(normalize=True)

    def distribution_distance(dist1, dist2):
        keys = set(dist1.keys()).union(dist2.keys())
        return sum(abs(dist1.get(k, 0) - dist2.get(k, 0)) for k in keys)

    eq_ids = create_equivalence_classes(df_temp, QI_cols)
    to_suppress = []
    for eq, group in df_temp.groupby(eq_ids):
        local_dist = group[sensitive_col].value_counts(normalize=True).to_dict()
        if distribution_distance(local_dist, global_dist.to_dict()) > t:
            to_suppress.append(eq)
    if not to_suppress:
        return df_temp
    else:
        mask = create_equivalence_classes(df_temp, QI_cols).apply(lambda x: x in to_suppress)
        df_temp = suppress_values(df_temp, mask, QI_cols)
        return df_temp

def k_anonymity_l_diversity(df, QI_cols, sensitive_col, k, l):
    """
    Combine k-anonymity and l-diversity.
    First enforce k-anonymity, then further suppress equivalence classes that do not satisfy l-diversity.
    """
    df_temp = k_anonymity(df, QI_cols, k)
    eq_ids = create_equivalence_classes(df_temp, QI_cols)
    diversity = df_temp.groupby(eq_ids)[sensitive_col].nunique()
    to_suppress = [eq for eq, d in diversity.items() if d < l]
    mask = create_equivalence_classes(df_temp, QI_cols).apply(lambda x: x in to_suppress)
    df_temp = suppress_values(df_temp, mask, QI_cols)
    return df_temp

def k_anonymity_t_closeness(df, QI_cols, sensitive_col, k, t):
    """
    Combine k-anonymity and t-closeness.
    First enforce k-anonymity, then further suppress equivalence classes that do not satisfy t-closeness.
    """
    df_temp = k_anonymity(df, QI_cols, k)
    global_dist = df[sensitive_col].value_counts(normalize=True)

    def distribution_distance(dist1, dist2):
        keys = set(dist1.keys()).union(dist2.keys())
        return sum(abs(dist1.get(k, 0) - dist2.get(k, 0)) for k in keys)

    eq_ids = create_equivalence_classes(df_temp, QI_cols)
    to_suppress = []
    for eq, group in df_temp.groupby(eq_ids):
        local_dist = group[sensitive_col].value_counts(normalize=True).to_dict()
        if distribution_distance(local_dist, global_dist.to_dict()) > t:
            to_suppress.append(eq)
    mask = create_equivalence_classes(df_temp, QI_cols).apply(lambda x: x in to_suppress)
    df_temp = suppress_values(df_temp, mask, QI_cols)
    return df_temp

In [ ]:
###############################################
# 6. Metrics
###############################################

def re_identification_risk(df, QI_cols):
    eq_ids = create_equivalence_classes(df, QI_cols)
    sizes = get_eq_class_sizes(eq_ids)
    avg_size = np.mean(list(sizes.values()))
    return 1.0 / avg_size

def disclosure_risk(df, QI_cols, sensitive_col):
    eq_ids = create_equivalence_classes(df, QI_cols)
    diversity = df.groupby(eq_ids)[sensitive_col].nunique()
    return np.mean(diversity == 1)

def suppression_ratio(df, QI_cols):
    """
    Compute the suppression ratio: the fraction of cells in QI_cols that are suppressed.
    """
    total = df.shape[0] * len(QI_cols)
    count_suppressed = (df[QI_cols] == "Suppressed").sum().sum()
    return count_suppressed / total

def distortion_numeric(df_original, df_anonymized, column, midpoints):
    """
    Compute distortion for a numeric QI column as the average absolute difference
    between the original numeric value and the corresponding midpoint of the anonymized category.
    Ignore rows where the value is "Suppressed".
    """
    valid = df_anonymized[column][df_anonymized[column] != "Suppressed"].astype(str)
    mapped = valid.map(midpoints)
    diff = (df_original.loc[mapped.index, column] - mapped).abs()
    return diff.mean()

def combined_distortion(df_original, df_anonymized):
    """
    Compute a combined distortion metric for the numeric QI columns ("age" and "education_num")
    by averaging their individual distortions.
    """
    age_midpoints = {'Young': 12.5, 'Adult': 35, 'Senior': 55, 'Elderly': 82.5}
    edu_midpoints = {'Low': 3, 'Mid': 8, 'High': 12.5, 'Top': 15.5}
    age_dist = distortion_numeric(df_original, df_anonymized, 'age', age_midpoints)
    edu_dist = distortion_numeric(df_original, df_anonymized, 'education_num', edu_midpoints)
    distortions = [d for d in [age_dist, edu_dist] if d is not None and not np.isnan(d)]
    if distortions:
        return np.mean(distortions)
    else:
        return np.nan

def mutual_info_metric(df, QI_cols, sensitive_col):
    df_temp = df[QI_cols + [sensitive_col]].copy()
    for c in QI_cols:
        df_temp[c] = df_temp[c].astype('category')
    X = pd.get_dummies(df_temp[QI_cols], drop_first=True)
    y = df_temp[sensitive_col]
    mi = mutual_info_classif(X, y, discrete_features=True)
    return np.sum(mi)

In [ ]:
###############################################
# 7. Updated Model Performance Functions Using Random Forest
###############################################

def model_performance(df, QI_cols, sensitive_col):
    """
    Compute model performance on the anonymized dataframe using a Random Forest.
    The features used are only the QI columns.
    """
    df_temp = df.copy()
    X = pd.get_dummies(df_temp[QI_cols], drop_first=True)
    y = df_temp[sensitive_col]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    clf = RandomForestClassifier(n_estimators=100, random_state=42)
    clf.fit(X_train, y_train)
    preds = clf.predict(X_test)
    preds_proba = clf.predict_proba(X_test)[:, 1]  # Predicted probabilities for AUC
    metrics = {
        "Accuracy": accuracy_score(y_test, preds),
        "Precision": precision_score(y_test, preds, zero_division=0),
        "Recall": recall_score(y_test, preds),
        "F1-Score": f1_score(y_test, preds),
        "AUC": roc_auc_score(y_test, preds_proba),
    }
    return metrics


###############################################
# 7. Updated Discernibility & CAVG Metrics
###############################################

def discernibility_metric(df, QI_cols, k):
    """
    Compute the Discernibility Metric (DM) for the anonymized table T*:

      DM(T*) = ∑_{EQ: |EQ| ≥ k} |EQ|^2 + ∑_{EQ: |EQ| < k} (|T| * |EQ|)

    where |T| is the total number of records.
    """
    T = len(df)
    eq_ids = create_equivalence_classes(df, QI_cols)
    sizes = get_eq_class_sizes(eq_ids)
    DM = 0
    for eq_size in sizes.values():
        if eq_size >= k:
            DM += eq_size ** 2
        else:
            DM += T * eq_size
    return DM

def cavg_metric(df, QI_cols, k):
    """
    Compute the Average Equivalence Class Size (CAVG) metric:

      CAVG(T*) = |T| / ((# of EQs) * k)
    """
    T = len(df)
    eq_ids = create_equivalence_classes(df, QI_cols)
    num_eq = len(get_eq_class_sizes(eq_ids))
    return T / (num_eq * k)

In [ ]:
###############################################
# 8. Parameter Optimization (with tqdm)
###############################################


def find_optimal_k(df, QI_cols, k_values=range(15, 25), weight_dm=0.5, weight_cavg=0.5):
    """
    For each candidate k, enforce k-anonymity on the DataFrame and compute both the
    Discernibility Metric (DM) and the Average Equivalence Class Size (CAVG). Then, a
    composite score is computed as:

         composite_score = weight_dm * normalized_DM + weight_cavg * normalized_CAVG

    (using min–max normalization for DM and CAVG across the candidate k values).

    The candidate with the smallest composite score is chosen as the optimal k.

    Parameters:
      df         : The input DataFrame.
      QI_cols    : List of quasi-identifier columns used to create equivalence classes.
      k_values   : Iterable of candidate k values.
      weight_dm  : Weight for the DM metric.
      weight_cavg: Weight for the CAVG metric.

    Returns:
      best_k          : The optimal candidate k (with the smallest composite score).
      results         : A list of dictionaries for each candidate k with its metrics.
      composite_scores: The numpy array of composite scores for all candidate k values.
      results_df      : A Pandas DataFrame with columns 'k', 'DM', 'CAVG', and 'CompositeScore',
                        which is useful for visualization.
    """
    results = []

    # Loop through each candidate k, compute metrics, and store results.
    for k in tqdm(k_values, desc="Processing k values"):
        # Apply the k-anonymity transformation (assume k_anonymity is defined)
        df_k = k_anonymity(df, QI_cols, k)
        # Compute the two metrics (assume these functions are defined)
        dm = discernibility_metric(df_k, QI_cols, k)
        cavg = cavg_metric(df_k, QI_cols, k)
        results.append({'k': k, 'DM': dm, 'CAVG': cavg})

    # Extract DM and CAVG values from the results.
    dm_vals = np.array([r['DM'] for r in results])
    cavg_vals = np.array([r['CAVG'] for r in results])

    # Normalize DM and CAVG using min–max normalization.
    norm_dm = (dm_vals - dm_vals.min()) / (dm_vals.max() - dm_vals.min() + 1e-10)
    norm_cavg = (cavg_vals - cavg_vals.min()) / (cavg_vals.max() - cavg_vals.min() + 1e-10)

    # Compute the composite scores.
    composite_scores = weight_dm * norm_dm + weight_cavg * norm_cavg

    # Choose the candidate with the smallest composite score.
    best_index = np.argmin(composite_scores)
    best_k = results[best_index]['k']

    # Package all the computed results into a DataFrame for easy plotting.
    results_df = pd.DataFrame({
        'k': [r['k'] for r in results],
        'DM': dm_vals,
        'CAVG': cavg_vals,
        'CompositeScore': composite_scores
    })

    return best_k, results, composite_scores, results_df

def plot_dm_vs_k(results_df):
    """
    Creates an interactive Altair chart for the Discernibility Metric (DM) vs. k,
    with the y-axis formatted in scientific (1e) notation.

    Parameters:
      results_df: A DataFrame with columns 'k' and 'DM'.

    Returns:
      An Altair Chart object.
    """
    chart = alt.Chart(results_df).mark_line(point=True).encode(
        x=alt.X('k:Q', title='k'),
        y=alt.Y('DM:Q', title='Discernibility Metric',
                axis=alt.Axis(format=".1e")),  # Use scientific notation (e.g., 1.2e+03)
        tooltip=['k', 'DM']
    ).properties(
        title='Discernibility Metric vs. k',
        width=600  # Make the chart wider.
    )

    return chart


def plot_cavg_vs_k(results_df):
    """
    Creates an interactive Altair chart for the Average Equivalence Class Size (CAVG) vs. k.

    Parameters:
      results_df: A DataFrame with columns 'k' and 'CAVG'.

    Returns:
      An Altair Chart object.
    """
    chart = alt.Chart(results_df).mark_line(point=True, color='orange').encode(
        x=alt.X('k:Q', title='k'),
        y=alt.Y('CAVG:Q', title='Average Equivalence Class Size'),
        tooltip=['k', 'CAVG']
    ).properties(
        title='CAVG vs. k',
        width=600  # Set the chart width to 600 pixels.
    )

    return chart



def plot_composite_vs_k(results_df, best_k):
    """
    Creates an interactive Altair chart for the Composite Score vs. k and overlays a vertical rule
    indicating the optimal k.

    Parameters:
      results_df: A DataFrame with columns 'k' and 'CompositeScore'.
      best_k    : The optimal k value determined from the composite scores.

    Returns:
      An Altair Chart object.
    """
    # Base composite score chart.
    chart = alt.Chart(results_df).mark_line(point=True, color='green').encode(
        x=alt.X('k:Q', title='k'),
        y=alt.Y('CompositeScore:Q', title='Composite Score'),
        tooltip=['k', 'CompositeScore']
    ).properties(
        title='Composite Score vs. k',
        width=600  # Set the chart width to 600 pixels.
    )

    # Vertical rule to indicate the optimal k.
    rule = alt.Chart(pd.DataFrame({'k': [best_k]})).mark_rule(
        color='purple', strokeDash=[5, 5]
    ).encode(
        x=alt.X('k:Q')
    )

    # Layer the rule over the composite score chart.
    chart_layered = chart + rule

    return chart_layered





def find_optimal_l(df, QI_cols, sensitive_col, l_values=range(2, 5)):
    """
    For each candidate l, enforce l-diversity and compute DM using a demo k value.
    Choose the l with minimal DM.
    """
    best_l, best_dm = None, float('inf')
    k_demo = min(range(15, 25))
    for l in tqdm(l_values, desc="Processing l values"):
        df_l = l_diversity(df, QI_cols, sensitive_col, l)
        dm = discernibility_metric(df_l, QI_cols, k_demo)
        if dm < best_dm:
            best_dm = dm
            best_l = l
    return best_l

def find_optimal_t(df, QI_cols, sensitive_col, t_values=[0.01, 0.05, 0.1, 0.2, 0.3]):
    """
    For each candidate t, enforce t-closeness and compute DM using a demo k value.
    Choose the t with minimal DM.
    """
    best_t, best_dm = None, float('inf')
    k_demo = min(range(15, 25))
    for t in tqdm(t_values, desc="Processing t values"):
        df_t = t_closeness(df, QI_cols, sensitive_col, t)
        dm = discernibility_metric(df_t, QI_cols, k_demo)
        if dm < best_dm:
            best_dm = dm
            best_t = t
    return best_t


In [ ]:
###############################################
# 9. Benchmark Function
###############################################

def benchmark_anonymization_methods(df_original, k_val, l_val, t_val, QI_cols, sensitive_col):
    """
    Apply each anonymization method with the given parameters, compute metrics,
    and return a summary table.
    """
    methods = [
        ("Original", None, []),  # Baseline: Fixed generalization without suppression.
        ("k-Anonymity", k_anonymity, [k_val]),
        ("l-Diversity", l_diversity, [l_val]),
        ("t-Closeness", t_closeness, [t_val]),
        ("k-Anonymity + l-Diversity", k_anonymity_l_diversity, [k_val, l_val]),
        ("k-Anonymity + t-Closeness", k_anonymity_t_closeness, [k_val, t_val])
    ]

    records = []
    for name, func, params in methods:
        df_anon = pd.DataFrame()
        if name == "Original":
            df_anon = generalize_QIs(df_original.copy())
        else:
            if name == "k-Anonymity":
                df_anon = func(df_original.copy(), QI_cols, params[0])
            elif name == "l-Diversity":
                df_anon = func(df_original.copy(), QI_cols, sensitive_col, params[0])
            elif name == "t-Closeness":
                df_anon = func(df_original.copy(), QI_cols, sensitive_col, params[0])
            elif name == "k-Anonymity + l-Diversity":
                df_anon = func(df_original.copy(), QI_cols, sensitive_col, params[0], params[1])
            else:  # "k-Anonymity + t-Closeness"
                df_anon = func(df_original.copy(), QI_cols, sensitive_col, params[0], params[1])

        model_metrics = model_performance(df_anon, QI_cols, sensitive_col)
        reid = re_identification_risk(df_anon, QI_cols)
        dm_val = discernibility_metric(df_anon, QI_cols, k_val)
        disc_risk = disclosure_risk(df_anon, QI_cols, sensitive_col)
        cavg_val = cavg_metric(df_anon, QI_cols, k_val)
        supp_ratio = suppression_ratio(df_anon, QI_cols)
        dist = combined_distortion(df_original, df_anon)
        mi_val = mutual_info_metric(df_anon, QI_cols, sensitive_col)

        rec = {
            "Method": name,
            "Re-ident. Risk": reid,
            "DM": dm_val if name == "k-Anonymity" or name == "k-Anonymity + l-Diversity" else "-",
            "Disclosure Risk": disc_risk if name == "Original" or name == "k-Anonymity" else "-",
            "CAVG": cavg_val  if name == "k-Anonymity" or name == "k-Anonymity + l-Diversity"  else "-",
            "Suppression Ratio": supp_ratio,
            "Combined Distortion": dist,
            "Mutual Info": mi_val,
            "Model Acc": model_metrics["Accuracy"],
            "Precision": model_metrics["Precision"],
            "Recall": model_metrics["Recall"],
            "F1-Score": model_metrics["F1-Score"],
            "AUC": model_metrics["AUC"],
        }
        records.append(rec)

    df_result = pd.DataFrame(records)
    df_result["Re-ident. Risk"] = df_result["Re-ident. Risk"].apply(lambda x: f"{x:.2e}" if pd.notnull(x) else x)
    df_result["DM"] = df_result["DM"].apply(
        lambda x: f"{x:.2e}" if isinstance(x, (int, float)) and pd.notnull(x) else x
    )
    return df_result

In [ ]:
df_original, QI_cols

(       age         workclass  fnlwgt   education  education_num  \
 0       39         State-gov   77516   Bachelors             13   
 1       50  Self-emp-not-inc   83311   Bachelors             13   
 2       38           Private  215646     HS-grad              9   
 3       53           Private  234721        11th              7   
 4       28           Private  338409   Bachelors             13   
 ...    ...               ...     ...         ...            ...   
 32556   27           Private  257302  Assoc-acdm             12   
 32557   40           Private  154374     HS-grad              9   
 32558   58           Private  151910     HS-grad              9   
 32559   22           Private  201490     HS-grad              9   
 32560   52      Self-emp-inc  287927     HS-grad              9   
 
            marital_status         occupation   relationship   race     sex  \
 0           Never-married       Adm-clerical  Not-in-family  White    Male   
 1      Married-civ-spou

In [ ]:
###############################################
# 10. Interactive Benchmarking with Start Button
###############################################

# Estimate optimal parameters using our heuristics.
optimal_k, results, composite_scores, results_df = find_optimal_k(df_original, QI_cols,range(10, 60))
optimal_l = 2
optimal_t = 0.5

Processing k values: 100%|██████████| 50/50 [01:11<00:00,  1.43s/it]


In [ ]:
print("Estimated optimal k =", optimal_k)
print("Estimated optimal l =", optimal_l)
print("Estimated optimal t =", optimal_t)

Estimated optimal k = 27
Estimated optimal l = 2
Estimated optimal t = 0.3


In [ ]:
chart_dm = plot_dm_vs_k(results_df)
chart_dm.display()  # or simply "chart_dm" in Jupyter Notebook


alt.Chart(...)

In [ ]:
chart_cavg = plot_cavg_vs_k(results_df)
chart_cavg.display()  # or simply "chart_cavg" in Jupyter Notebook


alt.Chart(...)

In [ ]:
chart_composite = plot_composite_vs_k(results_df, optimal_k)
chart_composite.display()  # or simply "chart_composite" in Jupyter Notebook


alt.LayerChart(...)

In [ ]:
def interactive_benchmark(k=optimal_k, l=optimal_l, t=optimal_t):
    df_bench = benchmark_anonymization_methods(df_original, k, l, t, QI_cols, sensitive_attribute)
    return df_bench

In [ ]:
# Create interactive sliders
slider_k = IntSlider(min=2, max=200, step=1, value=optimal_k, description='k')
slider_l = IntSlider(min=1, max=3, step=1, value=optimal_l, description='l')
slider_t = FloatSlider(min=0.0, max=1, step=0.01, value=optimal_t, description='t')

# Create a start button
start_button = Button(description="Start Benchmark", button_style='success')

result_data = []

# Define the button click event
def on_start_button_clicked(b):
    print(f"Running benchmark with k={slider_k.value}, l={slider_l.value}, t={slider_t.value}")
    result_df = interactive_benchmark(k=slider_k.value, l=slider_l.value, t=slider_t.value)
    display(result_df)
    result_data.append((result_df, slider_k.value, slider_l.value, slider_t.value))

# Attach the event to the button
start_button.on_click(on_start_button_clicked)

# Display the sliders and the button in a vertical layout
VBox([slider_k, slider_l, slider_t, start_button])


Running benchmark with k=27, l=2, t=0.5


,Method,Re-ident. Risk,DM,Disclosure Risk,CAVG,Suppression Ratio,Combined Distortion,Mutual Info,Model Acc,Precision,Recall,F1-Score,AUC
0,Original,1.99e-02,-,0.570988,-,0.000000,3.539096,0.163758,0.800860,0.653244,0.371738,0.473834,0.815545
1,k-Anonymity,4.42e-03,3.60e+07,0.118056,8.374743,0.091244,3.474079,0.154883,0.797021,0.653514,0.337365,0.445004,0.798191
2,l-Diversity,8.57e-03,-,-,-,0.095697,3.450119,0.284143,0.799785,0.650847,0.366645,0.469055,0.811065
3,t-Closeness,1.62e-02,-,-,-,0.138049,3.609697,0.518797,0.802549,0.657111,0.379376,0.481033,0.814909
4,k-Anonymity + l-Diversity,3.90e-03,4.87e+07,-,9.495771,0.143822,3.423560,0.193666,0.797021,0.653514,0.337365,0.445004,0.789945
5,k-Anonymity + t-Closeness,3.78e-03,-,-,-,0.214121,3.544033,0.289815,0.758790,0.000000,0.000000,0.000000,0.760600


Running benchmark with k=27, l=1, t=0.5


,Method,Re-ident. Risk,DM,Disclosure Risk,CAVG,Suppression Ratio,Combined Distortion,Mutual Info,Model Acc,Precision,Recall,F1-Score,AUC
0,Original,1.99e-02,-,0.570988,-,0.000000,3.539096,0.163758,0.800860,0.653244,0.371738,0.473834,0.815545
1,k-Anonymity,4.42e-03,3.60e+07,0.118056,8.374743,0.091244,3.474079,0.154883,0.797021,0.653514,0.337365,0.445004,0.798191
2,l-Diversity,1.99e-02,-,-,-,0.000000,3.539096,0.163758,0.800860,0.653244,0.371738,0.473834,0.815545
3,t-Closeness,1.62e-02,-,-,-,0.138049,3.609697,0.518797,0.802549,0.657111,0.379376,0.481033,0.814909
4,k-Anonymity + l-Diversity,4.42e-03,3.60e+07,-,8.374743,0.091244,3.474079,0.154883,0.797021,0.653514,0.337365,0.445004,0.798191
5,k-Anonymity + t-Closeness,3.78e-03,-,-,-,0.214121,3.544033,0.289815,0.758790,0.000000,0.000000,0.000000,0.760600


Running benchmark with k=27, l=3, t=0.5


...

,Method,Re-ident. Risk,DM,Disclosure Risk,CAVG,Suppression Ratio,Combined Distortion,Mutual Info,Model Acc,Precision,Recall,F1-Score,AUC
0,Original,1.99e-02,-,0.570988,-,0.000000,3.539096,0.163758,0.800860,0.653244,0.371738,0.473834,0.815545
1,k-Anonymity,8.02e-03,2.86e+07,0.206897,12.475479,0.035410,3.509902,0.160981,0.799478,0.654971,0.356461,0.461665,0.808918
2,l-Diversity,8.57e-03,-,-,-,0.095697,3.450119,0.284143,0.799785,0.650847,0.366645,0.469055,0.811065
3,t-Closeness,1.62e-02,-,-,-,0.138049,3.609697,0.518797,0.802549,0.657111,0.379376,0.481033,0.814909
4,k-Anonymity + l-Diversity,6.36e-03,3.87e+07,-,15.729952,0.105863,3.447680,0.243635,0.799478,0.654971,0.356461,0.461665,0.805149
5,k-Anonymity + t-Closeness,6.82e-03,-,-,-,0.166733,3.580108,0.392868,0.776294,0.552294,0.383195,0.452461,0.789719


Running benchmark with k=76, l=2, t=0.5


,Method,Re-ident. Risk,DM,Disclosure Risk,CAVG,Suppression Ratio,Combined Distortion,Mutual Info,Model Acc,Precision,Recall,F1-Score,AUC
0,Original,1.99e-02,-,0.570988,-,0.000000,3.539096,0.163758,0.800860,0.653244,0.371738,0.473834,0.815545
1,k-Anonymity,2.30e-03,6.31e+07,0.066667,5.712456,0.184454,3.411071,0.131480,0.791187,0.645517,0.297899,0.407666,0.779060
2,l-Diversity,8.57e-03,-,-,-,0.095697,3.450119,0.284143,0.799785,0.650847,0.366645,0.469055,0.811065
3,t-Closeness,1.62e-02,-,-,-,0.138049,3.609697,0.518797,0.802549,0.657111,0.379376,0.481033,0.814909
4,k-Anonymity + l-Diversity,2.15e-03,7.84e+07,-,6.120489,0.220786,3.372941,0.147995,0.791187,0.645517,0.297899,0.407666,0.769366
5,k-Anonymity + t-Closeness,2.00e-03,-,-,-,0.294340,3.484115,0.193902,0.758790,0.000000,0.000000,0.000000,0.724836


Running benchmark with k=40, l=2, t=0.6


,Method,Re-ident. Risk,DM,Disclosure Risk,CAVG,Suppression Ratio,Combined Distortion,Mutual Info,Model Acc,Precision,Recall,F1-Score,AUC
0,Original,1.99e-02,-,0.570988,-,0.000000,3.539096,0.163758,0.800860,0.653244,0.371738,0.473834,0.815545
1,k-Anonymity,3.35e-03,4.37e+07,0.110092,7.468119,0.124904,3.467765,0.138228,0.793183,0.646982,0.313813,0.422632,0.789332
2,l-Diversity,8.57e-03,-,-,-,0.095697,3.450119,0.284143,0.799785,0.650847,0.366645,0.469055,0.811065
3,t-Closeness,1.71e-02,-,-,-,0.129265,3.607902,0.498824,0.801935,0.656633,0.374920,0.477310,0.814725
4,k-Anonymity + l-Diversity,2.98e-03,5.82e+07,-,8.39201,0.172323,3.420937,0.160556,0.793183,0.646982,0.313813,0.422632,0.777820
5,k-Anonymity + t-Closeness,2.98e-03,-,-,-,0.235988,3.532590,0.243316,0.758329,0.440000,0.007002,0.013784,0.745336


In [ ]:
for i in result_data:
  i[0].to_csv(f"result_k{i[1]}_l{i[2]}_t{i[3]}.csv", index=False)

In [ ]:
###############################################
# 9. Usage Instructions
###############################################
#
# - Simply run this notebook (or script).
# - The code above loads and cleans the Adult dataset.
# - The "optimal_*" parameters are determined by simple naive heuristics
#   (Discernibility Metric elbow for k, minimal DM for l & t).
# - An interactive widget will appear (in Jupyter) to adjust k, l, and t.
# - The `interactive_benchmark` function will anonymize with each method
#   (k-Anonymity, l-Diversity, t-Closeness, their combos),
#   then compute and display the table of metrics.
#
# You can discuss:
# - How Re-identification Risk, Disclosure Risk, Attack Success rate
#   change as you raise k, l, or lower t.
# - How Info Loss, Mutual Info, or Model Performance degrade as we
#   generalize more heavily (increasing bins).
# - The trade-offs among these metrics.
#
# End of complete demonstration code.
###############################################